# 2.2 LangChain에서 도구(tool) 활용 방법
- LangChain에서 도구(tool)을 활용하는 방법을 알아봅니다
- 2.2에서는 도구를 활용하는 방법을 중점적으로 다루며, 도구를 활용한 에이전트 개발 방법은 2.3 강의에서 다룹니다

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

- [tool decorator](https://python.langchain.com/docs/how_to/custom_tools/#tool-decorator)를 사용하면 쉽게 도구를 만들 수 있습니다

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

- LLM을 호출했을 때와 도구를 사용했을 때의 차이를 알아봅니다

In [4]:
query = "3 곱하기 5는?"
llm.invoke(query)

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-D0ggjZN0GEIehlBB4iMkRmpcMEqbr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019be3fb-7074-7483-af6d-6d5bcd7d6380-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- 도구 리스트는 LLM에 해당하는 `BaseModel` 클래스에 `bind_tools` 메서드를 통해 전달합니다

In [5]:
llm_with_tools = llm.bind_tools([add, multiply])

- `AIMessage`의 `additional_kwargs` 속성은 `tool_calls`를 포함합니다
- `tool_calls`는 도구를 호출하는 메시지를 포함합니다

In [6]:
result = llm_with_tools.invoke(query)
result

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-D0gjLXT5qrqniU3vDVNcbGLUAOe7q', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019be3fd-ebcd-74d1-b2e9-f02003a33f7f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_uL1ZrC0my6BdAy9GSKQjdgCP', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'call_uL1ZrC0my6BdAy9GSKQjdgCP',
  'type': 'tool_call'}]

In [ ]:
from typing import Sequence
from langchain_core.messages import AnyMessage, HumanMessage

query = "3 x 5는?"
human_message = HumanMessage(query)
message_list: Sequence[AnyMessage] = [human_message]

In [9]:
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={})]

In [10]:
ai_message = llm_with_tools.invoke(message_list)
ai_message

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-D0glVQynfboDy2GOGcbobnN4xHM4j', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019be3ff-f30e-75f0-95cf-bc6c77d415f2-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_s1RPji0gSP1kHPt5glSaOD0g', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
ai_message.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'call_s1RPji0gSP1kHPt5glSaOD0g',
  'type': 'tool_call'}]

In [12]:
message_list.append(ai_message)

In [13]:
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-D0glVQynfboDy2GOGcbobnN4xHM4j', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019be3ff-f30e-75f0-95cf-bc6c77d415f2-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_s1RPji0gSP1kHPt5glSaOD0g', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 

- `AIMessage`의 `tool_calls`를 활용해서 도구를 직접 호출할 수도 있습니다

In [14]:
tool_message = multiply.invoke(ai_message.tool_calls[0])
tool_message

ToolMessage(content='15', name='multiply', tool_call_id='call_s1RPji0gSP1kHPt5glSaOD0g')

- 하지만 에이전트의 경우 도구를 직접 호출하는 것이 아니라 도구를 호출하는 메시지를 만들어서 전달합니다

In [15]:
message_list.append(tool_message)
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-D0glVQynfboDy2GOGcbobnN4xHM4j', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019be3ff-f30e-75f0-95cf-bc6c77d415f2-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_s1RPji0gSP1kHPt5glSaOD0g', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 

In [16]:
final_response = llm_with_tools.invoke(message_list)
final_response

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 109, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-D0gmsQAujpL6sy94Buv8SqeOKKjEN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019be401-4186-7a72-a303-e9dabcca2a78-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 109, 'output_tokens': 12, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
message_list.append(final_response)
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-D0glVQynfboDy2GOGcbobnN4xHM4j', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019be3ff-f30e-75f0-95cf-bc6c77d415f2-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_s1RPji0gSP1kHPt5glSaOD0g', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 